### IMPORTING LIBRARIES

In [67]:
import sys
import hashlib
import base64
from Cryptodome.Cipher import AES
from Cryptodome.Util.Padding import pad, unpad

### Zero-Width Characters for Text Steganography

In [68]:
ZERO_WIDTH_SPACE = "\u200B"  
ZERO_WIDTH_NON_JOINER = "\u200C"
ZERO_WIDTH_JOINER = "\u200D" 


In [69]:
AUTHORIZED_USERS = {"user" + str(i) for i in range(1, 21)}
ALL_USERS = {"user" + str(i) for i in range(1, 101)}

### Secure Password Hashing  function with SHA-256

In [70]:

def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()


### AES Encryption

In [71]:
def encrypt_message(secret_message, password):
    key = hashlib.sha256(password.encode()).digest()
    cipher = AES.new(key, AES.MODE_CBC)
    encrypted_bytes = cipher.encrypt(pad(secret_message.encode(), AES.block_size))
    return base64.b64encode(cipher.iv + encrypted_bytes).decode()


### Observations:  
The `encrypt_message` function uses AES-CBC encryption with a SHA-256-derived key and PKCS7 padding. A randomly generated IV is prepended to the encrypted message and base64-encoded for secure transmission. The notebook includes zero-width Unicode characters, suggesting text-based steganography for hiding encrypted messages. Additionally, user access control mechanisms (`AUTHORIZED_USERS` and `ALL_USERS`) indicate restricted access.  

### Conclusion:  
The encryption method is secure, but proper IV extraction is necessary for decryption. The use of zero-width characters implies a steganographic approach to conceal encrypted text. Combining AES encryption with text steganography enhances data security and secrecy. The presence of access control suggests that only authorized users can decrypt hidden messages.

### AES Decryption

In [72]:
def decrypt_message(encrypted_message, password):
    try:
        key = hashlib.sha256(password.encode()).digest()
        encrypted_data = base64.b64decode(encrypted_message)
        iv = encrypted_data[:16]
        cipher = AES.new(key, AES.MODE_CBC, iv)
        decrypted_bytes = unpad(cipher.decrypt(encrypted_data[16:]), AES.block_size)
        return decrypted_bytes.decode()
    except Exception:
        return None

### Observations:  
The `decrypt_message` function reverses the AES-CBC encryption process using a key derived from the SHA-256 hash of the provided password. It extracts the IV from the first 16 bytes of the base64-decoded encrypted message. The function then decrypts the remaining ciphertext and removes padding using `unpad`. Additionally, the `try-except` block ensures that decryption errors return `None`, preventing crashes.  

### Conclusion:  
The function securely decrypts messages if the correct password is provided, ensuring confidentiality. Proper IV extraction allows successful decryption of messages encrypted using `encrypt_message`. The use of padding removal ensures that the original plaintext is accurately reconstructed. The error handling mechanism prevents failures due to incorrect passwords or corrupted data.

###  Encode message using zero-width characters

In [90]:
def encode_message(carrier_text, secret_message, password):
    hashed_password = hash_password(password)
    encrypted_message = encrypt_message(secret_message, password)
    binary_secret = ''.join(format(ord(char), '08b') for char in encrypted_message)
    encoded = ''.join(ZERO_WIDTH_SPACE if bit == '0' else ZERO_WIDTH_NON_JOINER for bit in binary_secret)
    return carrier_text + ZERO_WIDTH_JOINER + hashed_password + encoded

### Observations:  
The `encode_message` function embeds an encrypted secret message within a carrier text using zero-width characters for steganography. It first hashes the password using `hash_password` and encrypts the secret message with `encrypt_message`. The encrypted message is then converted to binary, where `0s` are replaced with zero-width spaces and `1s` with zero-width non-joiners. Finally, the carrier text is appended with a zero-width joiner, the hashed password, and the encoded binary data.  

### Conclusion:  
This function effectively combines encryption with text-based steganography to conceal secret messages within normal-looking text. Using zero-width characters makes the hidden message visually undetectable while ensuring security through AES encryption. The inclusion of a hashed password helps in authentication, allowing only users with the correct password to retrieve the message. This method enhances both confidentiality and stealth in secure communication.

### Decode message for authorized users

In [91]:
def decode_message(stego_text):
    user_id = input("Enter your User ID: ")
    if user_id not in ALL_USERS:
        return "Invalid User ID."
    if user_id not in AUTHORIZED_USERS:
        return stego_text.split(ZERO_WIDTH_JOINER)[0]
    
    hashed_password_start = stego_text.find(ZERO_WIDTH_JOINER)
    if hashed_password_start == -1:
        return stego_text
    
    hashed_password_section = stego_text[hashed_password_start+1:]
    user_password = input("Enter password to access hidden message: ")
    hashed_password = hash_password(user_password)
    
    if hashed_password not in hashed_password_section:
        return stego_text.split(ZERO_WIDTH_JOINER)[0]
    
    encoded_part = hashed_password_section.split(hashed_password, 1)[1]
    binary_secret = ''.join('0' if char == ZERO_WIDTH_SPACE else '1' for char in encoded_part)
    encrypted_message = ''.join(chr(int(binary_secret[i:i+8], 2)) for i in range(0, len(binary_secret), 8))
    
    decrypted_message = decrypt_message(encrypted_message, user_password)
    if decrypted_message:
        return f"Correct Password! Hidden Message: {decrypted_message}"
    else:
        return "Incorrect password or corrupted data."

### Observations:  
The `decode_message` function extracts and decrypts a hidden message from a steganographic text. It first verifies the user’s ID against `ALL_USERS` and `AUTHORIZED_USERS`. If unauthorized, it returns the original carrier text. It then extracts the hashed password from the text and checks if the user-provided password matches. If valid, it decodes the zero-width character sequence back into binary, reconstructs the encrypted message, and attempts decryption using `decrypt_message`. Error handling ensures that incorrect passwords or corrupted data do not cause crashes.  

### Conclusion:  
This function securely retrieves hidden messages while enforcing access control. The dual-layer security—user authentication and password verification—prevents unauthorized access. The use of zero-width characters ensures the hidden message remains invisible in plain sight. By handling incorrect passwords gracefully, it maintains robustness and prevents information leakage to unauthorized users.

In [92]:
carrier_text = ("In the heart of a bustling city, where skyscrapers kissed the clouds and the streets buzzed with life, "
           "a story was unfolding. The sun cast long shadows as people hurried to their destinations, unaware of the "
           "hidden messages that surrounded them. Every building, every street sign, and every whisper of the wind carried "
           "secrets waiting to be uncovered. It was a world of coded words, of secrets hidden in plain sight, where only the "
           "keenest minds could decipher the truth. Amidst this, a special message lay embedded within, waiting for the "
           "right eyes to see it.")
secret_message = "Confidential Data: 12345XYZ"
password = "SecureKey"

In [93]:
stego_text = encode_message(carrier_text, secret_message, password)
print("Stego Text (Visible Part):", carrier_text)

Stego Text (Visible Part): In the heart of a bustling city, where skyscrapers kissed the clouds and the streets buzzed with life, a story was unfolding. The sun cast long shadows as people hurried to their destinations, unaware of the hidden messages that surrounded them. Every building, every street sign, and every whisper of the wind carried secrets waiting to be uncovered. It was a world of coded words, of secrets hidden in plain sight, where only the keenest minds could decipher the truth. Amidst this, a special message lay embedded within, waiting for the right eyes to see it.


In [96]:
retrieved_message = decode_message(stego_text)
print(retrieved_message)


Enter your User ID:  user2
Enter password to access hidden message:  SecureKey


Correct Password! Hidden Message: Confidential Data: 12345XYZ
